<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/8.Machine_Learning/descriptores/5.Ejercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 0) Cargar una de las imágenes histológicas

In [ ]:
import matplotlib.pyplot as plt
from skimage import io
import numpy as np
import cv2
from skimage.measure import label
from skimage import morphology
from skimage.measure import regionprops
from skimage.morphology import convex_hull_image
from scipy.ndimage.morphology import binary_fill_holes as bfh
from tabulate import tabulate
import math

In [ ]:
# Utilizar la librería skimage.io para leer la imagen 'histo_x.jpg' en formato RGB.
img = io.imread('images/histo_1.jpg')
# Normalizar la imagen para que los píxeles se encuentren en el rango [0, 1]
img_norm = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
# Visualizar la imagen
plt.imshow(img_norm, cmap='gray')
plt.show()

#### 1) Realizar una transformación de color para convertir la imagen al espacio de color CMYK

In [ ]:
def convert_to_CMYK(rgb_p):
    with np.errstate(invalid='ignore', divide='ignore'):
        K = 1 - np.max(rgb_p, axis=2)
        C = (1-rgb_p[:,:,0] - K)/(1-K)
        M = (1-rgb_p[:,:,1] - K)/(1-K)
        Y = (1-rgb_p[:,:,2] - K)/(1-K)
    CMYK = (np.dstack((C,M,Y,K)))
    return CMYK

In [ ]:
# Visualizar la imagen del canal magenta
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
rgb_p = rgb.astype('uint8')/255
CMYK = convert_to_CMYK(rgb_p)
C,M,Y,K = cv2.split(CMYK)
plt.imshow(M, cmap='gray') # Ploteamos Canal Magenta
plt.title('Imagen magenta')
plt.show()

#### 2) Umbralizar la imagen para separar los píxeles del fondo de la región tisular

Aplicar un filtro gaussiano de tamaño 5x5 y después utilizar el método de Otsu de manera que los píxeles correspondientes al lumen y al background de la imagen sean 1s y el resto de los píxeles tengan un valor de 0.

In [ ]:
img_gaus = cv2.GaussianBlur(M, (5,5), 0)
img_gaus = (img_gaus * 255).astype('uint8')
t, mask = cv2.threshold(img_gaus,0,1,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
plt.imshow(mask, cmap='gray')
plt.title('Máscara Otsu t=' + str(t))
plt.show()

#### 3) Limpiar la imagen eliminando los artefactos de lumen (objetos blancos pequeños que no son lúmenes)

Utilizar la librería skimage.morphology.remove_small_objects para eliminar aquellos objetos cuya área sea menor a 300 píxeles.

Más información en https://scikit-image.org/docs/dev/api/skimage.morphology.html#skimage.morphology.remove_small_objects

In [ ]:
mask = morphology.remove_small_objects(np.array(mask) == 1, min_size=300)
plt.imshow(mask, cmap='gray')
plt.title('Mascara sin hoyos')
plt.show()

#### 4) Rellenar con 0s el fondo de la imagen para quedarnos únicamente con los lúmenes

Aplicar el algoritmo de expansión a partir de semillas (region growing) de manera que únicamente los lúmenes sean blancos y el resto de la imagen negra.  
>Nota: Se pueden fijar las semillas de manera manual.

In [ ]:
new_mask = mask.copy().astype('uint8')
h, w = mask.shape
ref = np.zeros((h+2, w+2), np.uint8)

cv2.floodFill(new_mask, ref, (0,0), 0);
cv2.floodFill(new_mask, ref, (800,800), 0);

plt.imshow(new_mask, cmap='gray')
plt.title('Mascara sin fondo')
plt.show()

#### 5) Rellenar los objetos de los lúmenes

Rellenar los lúmenes con la función binary_fill_holes de la librería `scipy.ndimage.morphology`

In [ ]:
bordes = new_mask.copy()
filled_bordes = bfh(bordes)
plt.imshow(filled_bordes.astype('uint8'), cmap='gray')
plt.title('cleaned lumen mask')
plt.show()

Separamos los objetos

In [ ]:
lab, num = label(bordes, return_num=True)
print('número de objetos: ', num)
v,c = np.unique(lab, return_counts=True)
print('posibles valores de intensidad: ', v)
plt.imshow(lab, cmap='gray')
plt.show()

#### 6) Detectar y dibujar los contornos de los lúmenes sobre la imagen original

Dibujar los contornos de los lúmenes en color verde sobre la imagen original RGB. 

In [ ]:
imagen = img.copy()
for i in range(1, num+1): 
    objeto = lab == i
    objeto = objeto.astype('uint8')
    
    # covex hull
    convex_image = convex_hull_image(objeto)
    convex_image = convex_image.astype('uint8')
    conts,_ = cv2.findContours(convex_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    imagen = cv2.drawContours(imagen, conts, -1, (0,255,0), 10)

plt.imshow(imagen, cmap='gray')
plt.show()

#### 7) Extraer 13 características geométricas que permitan caracterizar el lumen recortado 

In [ ]:
# Calcular las siguientes características del crop del lumen de mayor área, redondeando su valor hasta el cuarto decimal.
# 1) Área
# 2) Área de la bounding box
# 3) Área convexa
# 4) Exentricidad
# 5) Diámetro equivalente
# 6) Extensión
# 7) Diámetro Feret
# 8) Longitud del eje mayor
# 9) Longitud del eje menor
# 10) Orientación
# 11) Perímetro
# 12) Solidez
# 13) Compacidad

# Determinar cuál es el lumen de mayor área y hacer un crop del mismo sobre la imagen original RGB.
# Visualizar el lumen cropeado.
new_lab, new_num = label(bordes, return_num=True)

# Extraemos las característicias geométricas
headers = v
A,BB,CA,E,ED,EX,MA,MiA,OR,P,S,CO,R = ['area'], ['bbox_area'], ['convex_area'], ['eccentricity'], ['equiv_diameter'], \
['extent'], ['major_axis'], ['minor_axis'], ['orientation'], ['perimeter'], ['solidity'], ['compactness'], ['rectangularity']

for i in range(1,new_num+1):
    objeto = new_lab == i
    prop = regionprops(objeto.astype(np.uint8))
    
    A.append(np.round(prop[0].area, 4))
    BB.append(np.round(prop[0].bbox_area, 4))
    CA.append(np.round(prop[0].convex_area, 4))
    E.append(np.round(prop[0].eccentricity, 4))
    ED.append(np.round(prop[0].equivalent_diameter, 4))
    EX.append(np.round(prop[0].extent, 4))
    MA.append(np.round(prop[0].major_axis_length, 4))
    MiA.append(np.round(prop[0].minor_axis_length, 4))
    OR.append(np.round(prop[0].orientation, 4))
    P.append(np.round(prop[0].perimeter, 4))
    S.append(np.round(prop[0].solidity, 4))
    CO.append(np.round(4*math.pi*prop[0].area/prop[0].perimeter**2, 4))
    R.append(np.round(prop[0].area/prop[0].bbox_area, 4))


my_data = [tuple(A), tuple(BB), tuple(CA), tuple(E), tuple(ED), tuple(EX), \
          tuple(MA), tuple(MiA), tuple(OR), tuple(P), tuple(S), tuple(CO), tuple(R)]

print(tabulate(my_data, headers=headers))

In [ ]:
objeto = lab == 8
objeto = objeto.astype('uint8')

convex_image = convex_hull_image(objeto)
convex_image = convex_image.astype('uint8')
plt.imshow(convex_image, cmap='gray')
plt.show()

conts,_ = cv2.findContours(convex_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # Encontramos los contornos en una máscara 
imagen = cv2.drawContours(img.copy(), conts, -1, (0,255, 128), 10) # Dibujamos los contornos
                     
plt.imshow(imagen, cmap='gray')
plt.show()